In [8]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from plotly.subplots import make_subplots
import plotly.express as px
import pickle


def create_merged_true_predicted_table(algos, runtimes, dim, ML_algo):
    for runtime in runtimes:
        df_merged = None
        for algo in algos:
            df = pd.read_csv(f"./data/processed_data/performance_data_regression_log/{algo}_{dim}D_{runtime}.csv", index_col=0)
            df.columns = [f'{algo}_true']
            df_merged = df if df_merged is None else df_merged.join(df)
        selector_folder = f"./results/classification/{ML_algo}/selector/"
        if not os.path.exists(selector_folder):
            os.makedirs(selector_folder)
        df_merged.to_csv(f"./results/classification/{ML_algo}/selector/as_{dim}D_{runtime}.csv")

def create_selector(selected_algo_portfolio, runtimes, dim, ML_algo):
    for runtime in runtimes:
        algos = selected_algo_portfolio[runtime]
        df = pd.read_csv(f"./results/classification/{ML_algo}/selector/as_{dim}D_{runtime}.csv", index_col=0)
        df['true_best_algo'] = df.apply(lambda row: algos[np.argmin([row[f'{algo}_true']for algo in algos])], axis=1)
        df['true_best_algo_value'] = df.apply(lambda row: np.min([row[f'{algo}_true']for algo in algos]), axis=1)
        df_predicted = pd.read_csv(f"./results/classification/{ML_algo}/predictions/{dim}D_{runtime}.csv", index_col=0)
        df = df.join(df_predicted)
        df.rename(columns={'predicted': 'predicted_best_algo'}, inplace=True)
        # df['predicted_best_algo'] = df.apply(lambda row: algos[np.argmin([row[f'{algo}_predicted']for algo in algos])], axis=1)
        df['true_value_predicted_best_algo'] = df.apply(lambda row: row[row.predicted_best_algo+"_true"], axis=1)
        for algo in algos:
            df[algo] = df.apply(lambda row: np.abs(row[f'{algo}_true']-row.true_best_algo_value), axis=1)
        df['AS-classification'] = df.apply(lambda row: np.abs(row.true_value_predicted_best_algo-row.true_best_algo_value), axis=1)
        df.to_csv(f"./results/classification/{ML_algo}/selector/as_{dim}D_{runtime}.csv")

def plot_boxplots(selected_algo_portfolio, runtimes, dim, ML_algo):

    for runtime in runtimes:
        algos = selected_algo_portfolio[runtime]
        df = pd.read_csv(f"./results/classification/{ML_algo}/selector/as_{dim}D_{runtime}.csv", index_col=0)
        last_n_column  = df.iloc[: , -(len(algos)+1):]
        fig = px.box(last_n_column, 
                    labels={"variable": "",  "value": "Loss"}, 
                    title=runtime, 
                    color = 'variable', 
                    color_discrete_sequence = px.colors.sequential.haline
                    )
        fig.update_layout(
            title_text=f'Runtime: {runtime}', title_x=0.5,
            autosize=False,
            width=1000,
            height=700,
            margin=dict(
                l=50,
                r=50,
                b=0,
                t=50,
                pad=4
            ),
            showlegend=False,
            font=dict(
                size=22
            )
        )
        # fig.write_image('./figures/boxplot_'+metric+".png")
        fig.show()
       

In [9]:
algos = ["DE", "DiagonalCMA", "modcma", "modde", "NaiveIsoEMNA", "NGOpt14", "NGOpt38", "OnePlusOne", "PSO", "RandomSearch", "RCobyla"]
dim = 5
runtimes = [100, 250, 500, 1000, 2500, 5000, 10000]
ML_algo = "RForest"
with open("./selected_algo_portfolio.pkl", "rb") as f:
    selected_algo_portfolio = pickle.load(f)

print(selected_algo_portfolio)

create_merged_true_predicted_table(algos, runtimes, dim, ML_algo)
create_selector(selected_algo_portfolio, runtimes, dim, ML_algo)
plot_boxplots(selected_algo_portfolio, runtimes, dim, ML_algo)

{100: ['OnePlusOne', 'RCobyla', 'modcma', 'NGOpt14'], 250: ['OnePlusOne', 'modcma', 'RCobyla', 'NGOpt14', 'DiagonalCMA'], 500: ['NGOpt14', 'modcma', 'OnePlusOne', 'RCobyla', 'DiagonalCMA'], 1000: ['NGOpt14', 'modcma', 'OnePlusOne', 'DiagonalCMA', 'DE'], 2500: ['NGOpt14', 'modcma', 'DE', 'OnePlusOne', 'DiagonalCMA'], 5000: ['NGOpt14', 'modcma', 'DE', 'OnePlusOne', 'DiagonalCMA', 'modde'], 10000: ['NGOpt14', 'modde', 'modcma', 'NGOpt38', 'OnePlusOne', 'DiagonalCMA']}
